# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
import json
from os import listdir, path
import numpy as np
from sklearn import svm, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report


ROBOT_POSE_DATA_DIR = 'robot_pose_data_json'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)[1:]
print(classes)
# Map class names to numerical labels
class_to_label = {cls_name: idx for idx, cls_name in enumerate(classes)}
label_to_class = {idx: cls_name for cls_name, idx in class_to_label.items()}
print(class_to_label)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']
{'Back': 0, 'Belly': 1, 'Crouch': 2, 'Frog': 3, 'HeadBack': 4, 'Knee': 5, 'Left': 6, 'Right': 7, 'Sit': 8, 'Stand': 9, 'StandInit': 10}


In [5]:
# got ValueError: could not convert string to float when i tried to load pickle files, i ahve limited time so decided to proceed with the json data. 
def load_pose_data():
    all_data = []
    all_labels = []
    for pose in classes:
        label = class_to_label[pose]
        filename = path.join(ROBOT_POSE_DATA_DIR, pose)
        try:
            with open(filename, 'r') as file:
                data = json.load(file)
                all_data.extend(data)
                all_labels.extend([label] * len(data))
        except Exception as e:
            print(f"Error loading {filename}: {e}")
            continue
    return all_data, all_labels

all_data, all_labels = load_pose_data()
print('Total number of data samples:', len(all_data))

#all_data


Total number of data samples: 222


In [6]:
# Convert data and labels to NumPy arrays
all_data = np.array(all_data, dtype=np.float32)
all_labels = np.array(all_labels, dtype=np.int32)

# one hot encoded lables (vectors)
num_classes = len(classes)
all_labels_categorical = to_categorical(all_labels, num_classes=num_classes)

# normalize data 
scaler = StandardScaler()
all_data = scaler.fit_transform(all_data)

# suffle and split
permutation = np.random.permutation(len(all_data))
all_data = all_data[permutation]
all_labels_categorical = all_labels_categorical[permutation]

# Split data into training and testing sets (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(
    all_data, all_labels_categorical, test_size=0.3, random_state=42)



## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [8]:
# network architecture
expected_lenght = 10
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(expected_lenght,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# model has to be complied befor you can use it 
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



C:\Users\Julian\RoboCup2024\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [11]:
# the acutal learning pahse (training) 20 epoch same result as up to 50 
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=16,
                    validation_split=0.2)

# model evaluation
#test_acc = model.evaluate(X_test, y_test, verbose=2)[1]
#print('\nTest accuracy:', test_acc)



Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9889 - loss: 0.8413 - val_accuracy: 0.9677 - val_loss: 0.9067
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9704 - loss: 0.7377 - val_accuracy: 0.9677 - val_loss: 0.8137
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9818 - loss: 0.6553 - val_accuracy: 0.9355 - val_loss: 0.7154
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9707 - loss: 0.5426 - val_accuracy: 0.9677 - val_loss: 0.6302
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.4703 - val_accuracy: 0.9677 - val_loss: 0.5563
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.3934 - val_accuracy: 0.9677 - val_loss: 0.4902
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.3373 - val_accuracy: 0.9677 - val_loss: 0.4320
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.2901 - val_accuracy: 0.9677 - val_loss: 0.3812
Epoch 9

### predicting

clf.predict(all_data[-1]), all_target[-1]

def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

expected = []
predicted = []
# YOUR CODE HERE

evaluate(expected, predicted)

## 4. Evaluate on the test data

In [12]:
# Existing code to generate y_true_classes and y_pred_classes
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Specify all class labels
labels = list(range(num_classes))  # num_classes = 11

# Generate the classification report
print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes, labels=labels, target_names=classes))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Classification Report:
              precision    recall  f1-score   support

        Back       1.00      1.00      1.00        11
       Belly       1.00      1.00      1.00         3
      Crouch       0.90      1.00      0.95         9
        Frog       1.00      1.00      1.00         6
    HeadBack       1.00      1.00      1.00         2
        Knee       1.00      1.00      1.00         1
        Left       1.00      1.00      1.00         4
       Right       1.00      0.75      0.86         4
         Sit       1.00      1.00      1.00         9
       Stand       1.00      1.00      1.00         3
   StandInit       1.00      1.00      1.00        15

    accuracy                           0.99        67
   macro avg       0.99      0.98      0.98        67
weighted avg       0.99      0.99      0.98        67



expected = []
predicted = []
# YOUR CODE HERE

evaluate(expected, predicted)

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

In [13]:
# save the model to ROBOT_POSE_CLF/clfv1.h5
MODEL_SAVE_PATH = 'robot_pose-clf/'

# Save the entire model to a file
model.save(MODEL_SAVE_PATH +'clfv1.keras')

# load the model
# the model should be enhanced in terms speed in edge devices ?? #TODO 
loaded_model = keras.models.load_model(MODEL_SAVE_PATH+'clfv1.keras')


Then, in the application we can load the trained classifier again.

clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]